# [1] Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.

Number of reviews: 568,454
Number of users: 256,059
Number of products: 74,258
Timespan: Oct 1999 - Oct 2012
Number of Attributes/Columns in data: 10

Attribute Information:

Id
ProductId - unique identifier for the product
UserId - unqiue identifier for the user
ProfileName
HelpfulnessNumerator - number of users who found the review helpful
HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
Score - rating between 1 and 5
Time - timestamp for the review
Summary - brief summary of the review
Text - text of the review
Objective:

# OBJECTIVE

Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).


[Q] How to determine if a review is positive or negative?



[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.

# Loading the data

he dataset is available in two forms

1.csv file
2.SQLite Database
In order to load the data, We have used the SQLITE dataset as it easier to query the data and visualise the data efficiently.

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score id above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [26]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [30]:
#importinhg the data
df = pd.read_csv("E:\\applied Ai\\data sets\\amazon fine food review analysis datsets\\Reviews.csv")


In [31]:
#top5 entries of data set
print(df.head())
#shape of df: no of rows, no. of features
print( df.shape)

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [32]:
#removing the entries or observation from the data set whose score value ==3.
df1 = df[df['Score'] !=3]
filtered_data = df1



In [33]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(10)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,1,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,1,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,1,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,1,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,1,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


# Exploratory Data Analysis








It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data. Following is an example:

In [36]:
# before droping the duplicate entries w'e sort the data from product id as ascending order
sorted_data = filtered_data.sort_values("ProductId", axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')


In [37]:
#Deduplication of entries
# removing the duplicate entries which has same userid,time,text which cannot be possivle for a user to write different reviews at the same tiem11
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [38]:
# to check for some entries which has helpfulness numerator > helfulness denominator whic is noty possible:
final= final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
final.shape
# there are only two such entries

(364171, 10)

In [39]:
# to check the positive and negative entires
final["Score"].value_counts()

1    307061
0     57110
Name: Score, dtype: int64

In [40]:
#%computation of negative and positive entries
no_positive = final[final["Score"] == 1].shape[0]
no_negative = final[final["Score"] == 0].shape[0]
x_per = no_positive+no_negative
print("positive reviews are",(no_positive/x_per)*100,"%")
print("negative reviews are",(no_negative/x_per)*100,"%")



positive reviews are 84.3178067446337 %
negative reviews are 15.682193255366297 %


# OBSERVATION

till this point of obesrvation we have preprocessed data to encode positve and negative reviews ,to remove duplicate entries and 
obaatained reusult is that we have 84.31% and 15.68% entries ,positive and negative reviews respectively.

# Text Preprocessing: Stemming, stop-word removal and Lemmatization.

Hence in the Preprocessing phase we do the following in the order below:-

Begin by removing the html tags

Remove any punctuations or limited set of special characters like , or . or # etc.

Check if the word is made up of english letters and is not alpha-numeric

Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)

Convert the word to lowercase

Remove Stopwords

Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming

In [ ]:
print

In [41]:
import re 
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer 

In [42]:
# find text which contain html tags
i=0
for sent in final["Text"].values:
    if(len(re.findall('<.*?>' , sent))):
        print(i)
        print(sent)
        break;
    i+=1;


6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [43]:
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english'))#set of words
sno = nltk.stem.SnowballStemmer('english')#initialising the snowballstemmer

def cleanhtml(sentence):#funtion to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,' ',sentence)
    return cleantext
def cleanpunc(sentence):#function to clean the word of any puncatuation
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned  = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned
print(stop)
print('****************************')
print(sno.stem('tasty'))

{'no', 'ourselves', 'through', 'about', 'y', 't', 'will', 'each', 'same', "you're", 'yours', 'them', 'again', "weren't", 'at', 'her', 'which', 'doing', "you'll", 'there', 'ma', 'be', "you'd", 'below', 'more', 'own', 'haven', 'very', 'off', 'with', 'they', 'these', 'because', 'in', 'hasn', 'doesn', 've', 'has', "shan't", 'his', 'only', 'their', 'where', 'she', 'as', 'theirs', 'is', 'didn', 'herself', 'your', 'too', 'him', 'couldn', 'themselves', 'won', 'the', 'itself', 'its', 'into', 'some', 'further', 'those', 'then', 'between', 'what', 'while', 'or', 'can', 're', 'd', "isn't", 'but', 'this', 'ain', 's', "didn't", 'a', 'up', 'on', 'than', "aren't", 'to', 'during', 'having', 'weren', 'himself', 'it', 'from', "doesn't", "that'll", 'ours', 'being', 'most', 'don', 'does', 'myself', "hasn't", "she's", 'few', 'o', 'just', 'been', 'now', 'out', 'until', 'under', 'such', "couldn't", 'needn', 'of', "don't", 'against', "mustn't", "won't", "shouldn't", 'wasn', 'shan', 'before', "hadn't", 'over', 

In [44]:
#code for implementing step-by-step the checks mentioned in the pre-process
# this code takes a while to run as it needs to run on 500k sentences.
i=0;
strl = ' '
final_string = []
all_positive_words = [] #stop words from +ve reviews here 
all_negative_words = [] # stop words from-ve reviews here
s = ' '
for sent in final['Text'].values:
    filtered_sentence = []
    #print(sent);
    sent = cleanhtml(sent)#remove html tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s = (sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s) #list of all words use
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) # line of all words use
                else:
                    continue
            else:
                continue
#print(filtered sentnece)
strl = b" ".join(filtered_sentence)#fianl string of cleaned words
final_string.append(strl)
i+=1

                        
            

In [ ]:
print(filtered_sentence)

In [ ]:
print(all_positive_words)

# Bag of Words(Bow)

In [45]:
#bow
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['Text'].values)


MemoryError: 

In [ ]:
type(final_counts)


In [28]:
final_counts.get_shape()

NameError: name 'final_counts' is not defined

# Bi-grams and n-grams

why there is the need of bi-grams and n-grams

#bi-gram, tri-gram and n-gram

removing stop words like "not" should be avoided before building n-grams
count_vect = CountVectorizer(ngram_range=(1,2))


# obsevation 

from the above data it can be seen that the mosty most common positive and negative words overlapfor eg.'like could be used as 'n
'not like' etc.
so, it is good idea consider pairs of consequesnt words (bi-grms) or q sequence of ngrams

In [ ]:
#bi-gram,tri-gram,and n-gram 

#removing stop words like 'not ' should be avoided
count_vect = CountVectorizer(ngram_range= (1,2))# source code from scikit learn
final_bigram_counts  = count_vect.fit_transform(final['Text'].values)

In [ ]:
final_bigram_counts.get_shape()

# TF-IDF

In [ ]:
tf_idf_vect = TfidfVecorizer(ngram_range = (1,2)) 
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)


In [ ]:
final_tf_idf.get_shape()

In [ ]:
features = final_tf_idf.get_featur_names()
len(features)


tf-idf representaion is in sparse matrix

# Word 2 Vec

In [46]:
# Using Google News Word2Vectors

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W17SRFAzZPY
# you can comment this whole cell
# or change these varible according to your need

# is_your_ram_gt_16g=False
# want_to_use_google_w2v = False
# want_to_train_w2v = True

# if want_to_train_w2v:
#     # min_count = 5 considers only words that occured atleast 5 times
#     w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
#     print(w2v_model.wv.most_similar('great'))
#     print('='*50)
#     print(w2v_model.wv.most_similar('worst'))
    
# elif want_to_use_google_w2v and is_your_ram_gt_16g:
#     if os.path.isfile('GoogleNews-vectors-negative300.bin'):
#         w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
#         print(w2v_model.wv.most_similar('great'))
#         print(w2v_model.wv.most_similar('worst'))
#     else:
#         print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")


# converting sentence into vector using Average Word 2 Vec ,TfIdf-Word2-Vec

In [ ]:
#average word2vec
#compute average word2vec for each review
sent_vectors = [];#the avg word 2 vec for each sentence /review is stored in this empty list
for sent in list_of_sent:#for each review /sentence
    sent_vec = np.zeros(50)#as wordvectorsare of zerolength
    cnt_words = 0;#no.of words with a validvector in the sentence /revoiew
    for word in sent:#for each word in a review 
        try:
            vec = w2v_model.wv[word]
            sent_vec+= vec
            cnt_words +=1
        except:
            pass
    sent_vec/=cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

In [ ]:
#TF-IDF weigthed Word2vec
tfidf_feat = tfidf_vect.get_feature_names()#tfidf words/col-names
#final_tf_idf is tge sparse matrix with row = sentcecol = word

tfidf_sent_vectors = [];#tfidf-w2v for each srntnce is stoired
row =0;
for sent in list_of_sent:#for each review 
    sent_vec = np.zeros(50)#as wordvectorsare of zerolengt
    weight_sum = 0;#no. of words with the valid vector in the sentenvce
    for word in sent:# for each word in review
        try:
            vec = w2v_model.wv[word]
            tfidf = final_tf_idf[row,tfidf_feat.index(word)]
            sent_vec+=(vec*tf_idf)
            weight_sum+=tfidf
        except:
            pass
sent_vec/=weight_sum
tfidf_sent_vectors.append(sent_vec)
row+=1

        